In [1]:
'''导入库'''

'导入库'

In [2]:
!pip install torch torchvision torchaudio --ignore-installed TBB

ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'C:\\data\\Library\\bin\\tbbmalloc.dll'
Consider using the `--user` option or check the permissions.



  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d3/1d/a257913c89572de61316461db91867f87519146e58132cdeace3d9ffbe1f/torch-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/e4/c3/a21a75dd2de8114a6876f16a36b033e3e62f8ade68085a711b24f4b57c17/torchvision-0.18.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torchvision-0.18.1-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/7a/bf/5af72c1c4522bcf67df140427120b0e7898b2abc5afa5da917b722983a5b/torchaudio-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torchaudio-2.3.1-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Obtaining dependency information for TBB from https://files.pythonhosted.org/packages/7b/2d/1e1c70fae8ace27e6200fb71c2372a9aeac2baba474b160

In [3]:
import chardet

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


C:\data\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
'''numpy 和 pandas 用于数据处理。
torch 是 PyTorch 深度学习库。
Dataset 和 DataLoader 用于数据加载和批处理。
BertTokenizer 和 BertForSequenceClassification 来自 Hugging Face Transformers 库，用于使用预训练的 BERT 模型进行文本分类。
accuracy_score, precision_score, recall_score, f1_score 是评估模型性能的指标'''

'numpy 和 pandas 用于数据处理。\ntorch 是 PyTorch 深度学习库。\nDataset 和 DataLoader 用于数据加载和批处理。\nBertTokenizer 和 BertForSequenceClassification 来自 Hugging Face Transformers 库，用于使用预训练的 BERT 模型进行文本分类。\naccuracy_score, precision_score, recall_score, f1_score 是评估模型性能的指标'

In [6]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'token_type_ids': encoding['token_type_ids'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [7]:
def train_model(model, train_loader, test_loader, optimizer, criterion, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}')


In [9]:
def evaluate_model(model, test_loader):
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs)
            _, predicted = torch.max(outputs.logits, 1)
            test_preds.extend(predicted.tolist())
            test_targets.extend(labels.tolist())
    acc = accuracy_score(test_targets, test_preds)
    precision = precision_score(test_targets, test_preds, average='binary')
    recall = recall_score(test_targets, test_preds, average='binary')
    f1 = f1_score(test_targets, test_preds, average='binary')
    print(f'Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

In [ ]:
if __name__ == '__main__':
    # Parameters
    MAX_LENGTH = 100
    BATCH_SIZE = 32
    NUM_EPOCHS = 5
    LEARNING_RATE = 2e-5

    # Data
    dataset = pd.read_csv('./data/tieba_senti_10k.csv', delimiter=',', encoding='gbk')
    dataset['label'] = dataset['label'].astype(int)  # 将label转换为整数类型
    data = dataset['review'].tolist()
    labels = dataset['label'].tolist()

    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    train_dataset = TextDataset(X_train, y_train, tokenizer, MAX_LENGTH)
    test_dataset = TextDataset(X_test, y_test, tokenizer, MAX_LENGTH)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)
    model.to(device)

    # Loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    # Training
    train_model(model, train_loader, test_loader, optimizer, criterion, NUM_EPOCHS)

    # Evaluation
    evaluate_model(model, test_loader)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5, Loss: 0.1672
